In [1]:
#pip install selenium


In [8]:
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")

import time
from bs4 import BeautifulSoup
import urllib.request 
#driver = webdriver.Chrome()

In [9]:
def get_urls(companies):
    driver = webdriver.Chrome(options=chrome_options)
    #driver.Manage().Cookies.DeleteAllCookies()
    driver.get("https://lxrmarketplace.com/seo-domain-finder.html")

    driver.find_element(By.ID, "closeButton").click()
    driver.implicitly_wait(1)

    search_field = driver.find_element(By.NAME, "querySEOCompanyNames")

    #entering data into search field
    for company in companies:
        search_field.send_keys(company, Keys.ENTER)

    driver.implicitly_wait(1)

    search_button = driver.find_element(By.ID, "getSEODomainFinder")
    search_button.click()

    time.sleep(5)
    #html=driver.page_source
    #soup=BeautifulSoup(html,'html.parser')

    table = pd.read_html(driver.page_source)[0]   # pandas gets all the tables from page, we just need the first
    driver.quit()
    return table

In [10]:
def get_soup(unformatted_url):
    if unformatted_url[8:12]!='www.':
        html = urllib.request.urlopen("https://tracxn.com/d/companies/" + unformatted_url[8:])
        url="https://tracxn.com/d/companies/" + unformatted_url[8:]
        
    if unformatted_url[8:12]=="www.":
        html = urllib.request.urlopen("https://tracxn.com/d/companies/" + unformatted_url[12:])
        url="https://tracxn.com/d/companies/" + unformatted_url[12:]
    print(unformatted_url)
    print(url)
    #df = pd.DataFrame(pd.DataFrame(columns = ['Company Name','Location', 'Year', 'Stage', 'Total Funding', 'Valuation', 'Annual Revenue', 'Similar Companies']))
    #temp = pd.DataFrame()
    #html = urllib.request.urlopen(url)

    # parsing the html file
    htmlParse = BeautifulSoup(html, 'html.parser')
    return htmlParse

In [11]:
def get_data(htmlParse, startup_name):
    #startup = "HomeCapital"
    startup_row_df_columns = ['Company Name']
    startup_row_values = []
    for para in htmlParse.find_all('p'):
        if len(para.contents)==2 :
            if para.contents[0].text == "Share Holder":
                break
            else:
                startup_row_df_columns.append(para.contents[0].text)
                startup_row_values.append(para.contents[1].text)
    startup_row_df_columns, startup_row_values
    startup_row_df = pd.DataFrame(columns = startup_row_df_columns).set_index('Company Name')
    startup_row_df.loc[startup_name]=startup_row_values
    return startup_row_df

In [12]:
def final_table(final_data, url_table):
    return pd.merge(url_table, final_data, how='outer', on= 'Company Name')

In [13]:
def main():
    companies = ["Basic Home Loan", "Easiloan", "Garuda Aerospace", "HomeCapital"]
    final = pd.DataFrame()
    startup_row = pd.DataFrame()
    table = get_urls(companies)
    for index, startup in table.iterrows():
        startup_row = get_data(get_soup(startup['Domain / URL']), startup['Company Name'])
        final = pd.concat([final,startup_row])
    display(final)
    return final_table(final,table)

In [14]:
df = main()
df

https://basichomeloan.com
https://tracxn.com/d/companies/basichomeloan.com
https://www.easiloan.com
https://tracxn.com/d/companies/easiloan.com
https://www.garudaaerospace.com
https://tracxn.com/d/companies/garudaaerospace.com
https://homecapital.in
https://tracxn.com/d/companies/homecapital.in


,Founded Year,Location,Company Stage,Total Funding,Last Funding Round,Valuation,Employee Count,Similar Cos.,Annual Revenue
Company Name,,,,,,,,,
Basic Home Loan,2020,"Gurgaon, India",Series A,$4M,"$3.5M, Series A, Oct 04, 2021","$12.2M as on Nov 01, 2021","53 as on Mar 31, 2022 (View Historical Trend)","Better Mortgage, Tomo, Pretto, Trussle, Mojo M...",NaN
Easiloan,2020,India,Seed,$1.09M,"$1.09M, Seed, Feb 24, 2021","$1.23M as on Feb 24, 2021",NaN,"Better Mortgage, Tomo, Pretto, Trussle, Mojo M...",NaN
Garuda Aerospace,2015,"Chennai, India",Seed,$1.48M,"$984K, Seed, May 05, 2022","$28.7M as on Jun 01, 2022","7 as on Dec 31, 2021 (View Historical Trend)","Skydio, Dajiang Innovation Technology, Percept...","$245K as on Dec 31, 2020"
HomeCapital,2017,"Mumbai, India",Series B,$14.9M,"$7.22M, Series B, Sep 29, 2021","$26.9M as on Sep 29, 2021","21 as on Mar 31, 2022 (View Historical Trend)","Landed, HomeCapital, Proportunity, OnLadder, T...","$500K as on Dec 31, 2020"


,S.No.,Company Name,Domain / URL,Founded Year,Location,Company Stage,Total Funding,Last Funding Round,Valuation,Employee Count,Similar Cos.,Annual Revenue
0,1.0,Basic Home Loan,https://basichomeloan.com,2020,"Gurgaon, India",Series A,$4M,"$3.5M, Series A, Oct 04, 2021","$12.2M as on Nov 01, 2021","53 as on Mar 31, 2022 (View Historical Trend)","Better Mortgage, Tomo, Pretto, Trussle, Mojo M...",NaN
1,2.0,Easiloan,https://www.easiloan.com,2020,India,Seed,$1.09M,"$1.09M, Seed, Feb 24, 2021","$1.23M as on Feb 24, 2021",NaN,"Better Mortgage, Tomo, Pretto, Trussle, Mojo M...",NaN
2,3.0,Garuda Aerospace,https://www.garudaaerospace.com,2015,"Chennai, India",Seed,$1.48M,"$984K, Seed, May 05, 2022","$28.7M as on Jun 01, 2022","7 as on Dec 31, 2021 (View Historical Trend)","Skydio, Dajiang Innovation Technology, Percept...","$245K as on Dec 31, 2020"
3,4.0,HomeCapital,https://homecapital.in,2017,"Mumbai, India",Series B,$14.9M,"$7.22M, Series B, Sep 29, 2021","$26.9M as on Sep 29, 2021","21 as on Mar 31, 2022 (View Historical Trend)","Landed, HomeCapital, Proportunity, OnLadder, T...","$500K as on Dec 31, 2020"
